# MONAI-based 3D Brain Tumor Segmentation
## Optimized for Google Colab with A100 GPUs

This notebook implements a robust 3D brain tumor segmentation pipeline using MONAI framework, designed to handle common issues and work efficiently with BraTS data on Google Colab.

### Features:
- ✅ Fixed ROI size issues for smaller volumes
- ✅ Resolved tensor dimension mismatches
- ✅ Autocast compatibility fixes
- ✅ Support for .rar/.zip archive extraction
- ✅ Optimized for A100 GPU performance
- ✅ Robust training/validation pipeline

## 1. Setup and Installation

First, let's clone the repository and install dependencies:

In [ ]:
# Clone the repository
!git clone https://github.com/hrishikeshHD/BrainTumorSegmentation.git
%cd BrainTumorSegmentation

In [ ]:
# Run the Colab setup script
!python colab_setup.py

## 2. GPU and Environment Check

Let's verify that we have access to a GPU and check our environment:

In [ ]:
import torch
import monai
from monai.config import print_config

print(f"PyTorch version: {torch.__version__}")
print(f"MONAI version: {monai.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Print MONAI configuration
print_config()

## 3. Data Preparation

### Option A: Upload your own BraTS data

If you have BraTS data in .zip or .rar format, upload it to `/content/data/`:

In [ ]:
from google.colab import files
import os

# Create data directory
os.makedirs('/content/data', exist_ok=True)

# Upload files
print("Please upload your BraTS data (.zip or .rar files):")
uploaded = files.upload()

# Move uploaded files to data directory
for filename in uploaded.keys():
    os.rename(filename, f'/content/data/{filename}')
    print(f"Uploaded: {filename}")

### Extract Archives (if needed)

The segmentation script will automatically extract .zip and .rar files, but you can also do it manually:

In [ ]:
import zipfile
import rarfile
from pathlib import Path

data_path = Path('/content/data')

# Extract ZIP files
for zip_file in data_path.glob('*.zip'):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(data_path)
    print(f"Extracted: {zip_file.name}")

# Extract RAR files
for rar_file in data_path.glob('*.rar'):
    with rarfile.RarFile(rar_file, 'r') as rar_ref:
        rar_ref.extractall(data_path)
    print(f"Extracted: {rar_file.name}")

### Verify Data Structure

Let's check if the data is properly organized:

In [ ]:
import os
from pathlib import Path

def check_data_structure(base_path):
    base = Path(base_path)
    
    processed_data = base / 'processed_data'
    train_dir = processed_data / 'train'
    test_dir = processed_data / 'test'
    
    print(f"Checking data structure in: {base}")
    print(f"Processed data exists: {processed_data.exists()}")
    print(f"Train directory exists: {train_dir.exists()}")
    print(f"Test directory exists: {test_dir.exists()}")
    
    if train_dir.exists():
        train_cases = list(train_dir.iterdir())
        print(f"Training cases found: {len(train_cases)}")
        if train_cases:
            print(f"Example case: {train_cases[0].name}")
    
    if test_dir.exists():
        test_cases = list(test_dir.iterdir())
        print(f"Test cases found: {len(test_cases)}")
        if test_cases:
            print(f"Example case: {test_cases[0].name}")

check_data_structure('/content/data')

## 4. Configure Training Parameters

Adjust these parameters based on your data and requirements:

In [ ]:
# Training configuration optimized for A100 GPU
config = {
    "seed": 42,
    "max_epochs": 100,
    "batch_size": 2,  # Increase if you have more GPU memory
    "learning_rate": 1e-4,
    "weight_decay": 1e-5,
    "roi_size": (128, 128, 128),  # Will adapt to smaller volumes automatically
    "sw_batch_size": 4,
    "spacing": (1.0, 1.0, 1.0),
    "num_classes": 4,  # Background, NCR/NET, ED, ET
    "model_name": "unet",  # Options: unet, segresnet, unetr
    "use_amp": True,  # Automatic Mixed Precision for A100
    "cache_rate": 0.5,
    "num_workers": 4,
    "log_interval": 50,
}

print("Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 5. Start Training

Now let's run the brain tumor segmentation training:

In [ ]:
# Import the segmentation pipeline
from brain_tumor_segmentation import BrainTumorSegmentation, create_file_list
import logging

# Setup logging to see training progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the segmentation pipeline
segmentation = BrainTumorSegmentation(config)

# Setup data directories (will handle archive extraction automatically)
train_dir, test_dir = segmentation.setup_data_directories('/content/data')

print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")

In [ ]:
# Create file lists for training and validation
train_files = create_file_list(train_dir, "train")
val_files = create_file_list(test_dir, "test")

print(f"Training files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")

if train_files:
    print("\nExample training file:")
    print(train_files[0])
else:
    print("\n⚠️ No training files found! Please check your data structure.")
    print("Expected structure: processed_data/train/CaseName/CaseName_modality.nii.gz")

In [ ]:
# Start training (this will take a while)
if train_files:
    print("Starting training...")
    print("This may take several hours depending on your data size and epochs.")
    
    # Start the training process
    segmentation.train(train_files, val_files)
    
    print("\n✅ Training completed!")
    print("Best model saved as: best_metric_model.pth")
else:
    print("❌ Cannot start training without data. Please upload and organize your BraTS data first.")

## 6. Monitor Training (Optional)

You can monitor training progress and GPU usage:

In [ ]:
# Monitor GPU usage during training
!nvidia-smi

In [ ]:
# Check memory usage
import torch
import psutil

if torch.cuda.is_available():
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")

print(f"CPU Memory Usage: {psutil.virtual_memory().percent}%")
print(f"Available RAM: {psutil.virtual_memory().available / 1024**3:.2f} GB")

## 7. Results and Model Evaluation

After training, you can evaluate the model and visualize results:

In [ ]:
# Load the best trained model
import torch
from brain_tumor_segmentation import BrainTumorSegmentation

# Load model state
if os.path.exists('best_metric_model.pth'):
    print("✅ Best model found!")
    
    # Create model instance
    segmentation = BrainTumorSegmentation(config)
    model = segmentation.create_model()
    
    # Load trained weights
    model.load_state_dict(torch.load('best_metric_model.pth'))
    model.eval()
    
    print("Model loaded successfully!")
else:
    print("❌ No trained model found. Please run training first.")

## 8. Download Trained Model

Download the trained model for future use:

In [ ]:
from google.colab import files
import os

# Download the best model
if os.path.exists('best_metric_model.pth'):
    files.download('best_metric_model.pth')
    print("Model downloaded successfully!")
else:
    print("No model file found to download.")

## 9. Troubleshooting

Common issues and solutions:

### Memory Issues
- Reduce `batch_size` in config
- Reduce `roi_size` to (96, 96, 96) or (64, 64, 64)
- Set `cache_rate` to 0.1 or 0.2

### Data Loading Issues
- Check that your data follows BraTS naming convention
- Ensure files are in .nii.gz format
- Verify the folder structure matches: processed_data/{train|test}/

### GPU Issues
- Make sure you selected GPU runtime in Colab
- Check GPU availability with `torch.cuda.is_available()`

### Training Issues
- Start with fewer epochs for testing
- Monitor loss curves for convergence
- Adjust learning rate if needed